In [34]:
import keras
from tqdm import tqdm 
import tensorflow as tf 
from keras.applications.resnet50 import ResNet50
from keras.layers import Flatten, Input
from keras.models import Model
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
import numpy as np
from io import BytesIO
import time
import cv2
import keras
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Model, load_model
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras.optimizers import SGD
start = time.time()
from keras.applications import Xception
from keras.utils import multi_gpu_model

In [25]:
!pip install opencv-python

In [35]:
batch_size = 20
image_size = 224
num_class = 4

In [36]:
import sys
train_path = "/mnt/batch/tasks/shared/LS_root/mounts/clusters/vm-vaa001/code/users/vishal/Data3/Train"
test_path = "/mnt/batch/tasks/shared/LS_root/mounts/clusters/vm-vaa001/code/users/vishal/Data3/Test"

In [37]:
%pwd

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/vm-vaa001/code/users/vishal'

In [14]:
import os
from zipfile import ZipFile

In [38]:
os.listdir()

['.ipynb_checkpoints',
 'Data3',
 'Data3.zip',
 'VAA phase-1',
 'vaa-trainig01.ipynb']

In [16]:
with ZipFile('Data3.zip','r') as ZipObj:
    ZipObj.extractall()

In [17]:
!pwd

/mnt/batch/tasks/shared/LS_root/mounts/clusters/vm-vaa001/code/users/vishal


In [39]:
#Load the Resnet model
#base_model = VGG16(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))
b_model = ResNet50(weights='imagenet', pooling=max, include_top = False, input_shape=(image_size, image_size, 3) ) 


#print(base_model.summary())

    # Freeze the layers 
for layer in b_model.layers:
    layer.trainable = False
 
# ##Create the model
model = keras.models.Sequential()

# # Add the vgg convolutional base model
model.add(b_model)
 
# # Add new layers
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_class, activation='softmax'))
 
# # Show a summary of the model. Check the number of trainable parameters    
##print(model.summary())

ResourceExhaustedError: OOM when allocating tensor with shape[3,3,64,64] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:TruncatedNormal]

In [32]:
from keras.optimizers import SGD

model.compile(loss='categorical_crossentropy',
          optimizer=SGD(lr=1e-3),
          metrics=['accuracy'])

train_datagen = ImageDataGenerator(
         zoom_range=0.2, # randomly zoom into images
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images
 
validation_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(image_size, image_size),
        batch_size=batch_size,
        class_mode='categorical',
         shuffle = True)
 
validation_generator = validation_datagen.flow_from_directory(
        test_path,
        target_size=(image_size, image_size),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True)


Found 2000 images belonging to 4 classes.
Found 400 images belonging to 4 classes.


In [33]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=100,
      validation_data= validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size)
 
# Save the model
model.save('resnet_model.h5')

RuntimeError: You must compile your model before using it.